In [1]:
# import sys
# import os
# import numpy as np
# import pandas as pd
# import nibabel as nb
# import torch
# import matplotlib.pyplot as plt
# from torch.utils.data import Dataset, DataLoader
# sys.path.append('/host/d/Github')
# import Whole_heart_segmentation_junzhe.functions_collection as ff
# import Whole_heart_segmentation_junzhe.Data_processing as Data_processing
# import Whole_heart_segmentation_junzhe.Build_lists.Build_list as Build_list
# import Whole_heart_segmentation_junzhe.data_loader.random_aug as random_aug
# import Whole_heart_segmentation_junzhe.data_loader.generator as generator
# import Whole_heart_segmentation_junzhe.segment_anything.model as model
# import torch.nn.functional as F

# import torch.optim as optim
# from torch.cuda.amp import GradScaler
# from tqdm import tqdm  # 进度条


import sys
import os
# 获取项目根目录（自动适配你的路径）
root_dir = os.getcwd()  # 因为main.ipynb就在根目录里，所以当前目录就是根目录
sys.path.append(root_dir)  # 把根目录加入Python的搜索路径

# 后面的导入可以简化了
import numpy as np
import pandas as pd
import nibabel as nb
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# 简化导入（不用写全路径了）
import functions_collection as ff
import Data_processing
from Build_lists.Build_list import Build  # 精准导入Build_list.py里的Build类
import data_loader.random_aug as randaug
import data_loader.generator as generator
import segment_anything.model as model  # 直接从segment_anything导入model

import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import GradScaler
from tqdm import tqdm  # 进度条

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]


### step 1: define trial name

In [2]:
trial_name = 'trial'

### step 2: build patient list

In [3]:
# change the excel path to your own path
patient_list_spreadsheet = os.path.join('/host/d/Github/Whole_heart_segmentation_junzhe/example_data/Patient_list','patient_list.xlsx')
build_sheet = Build(patient_list_spreadsheet)
# build_sheet =  Build_lists.Build(patient_list_spreadsheet)
# train
_, patient_id_list_train, slice_index_list_train, img_file_list_train, seg_file_list_train = build_sheet.__build__(batch_list = [0])  # just as an example, use batch 0 for train
print('all train img files:', img_file_list_train)

# # define val
# _,_,input_file_val, reference_file_val = build_sheet.__build__(batch_list = [1])  # just as an example, use the same batch for val



all train img files: ['/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_0.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_1.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_2.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_3.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_4.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_5.nii.gz']


### data generator

In [4]:
# define this generator
generator_train = generator.Dataset_CMR(
    image_file_list = img_file_list_train,
    
    seg_file_list = seg_file_list_train,

    image_shape = [128,128],
    center_crop_according_to_which_class  = [1], #default: crop according to class 1 (LV)

    shuffle = True,
    image_normalization = True,
    augment = True,
    augment_frequency = 0.1,
    )

In [5]:
ds = generator_train
dl = DataLoader(ds, batch_size = 1, shuffle = False, pin_memory = True, num_workers = 0)# cpu_count())

In [6]:
# build model
import argparse
import os

def get_args_parser( vit_type = "vit_h", original_SAM_weights = None):
    parser = argparse.ArgumentParser('SAM fine-tuning', add_help=True)

    parser.add_argument('--resume', default = original_SAM_weights)

    parser.add_argument('--img_size', default=128, type=int) 

    parser.add_argument('--vit_type', default=vit_type, type=str, choices=[ 'vit_b', 'vit_l', 'vit_h'],)
    
 
    return parser

original_sam = os.path.join('/host/d/Data/SAM_weights','sam_vit_b_01ec64.pth')

args = get_args_parser(vit_type = "vit_b",original_SAM_weights = original_sam)
args = args.parse_args([])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
our_model = model.build_model(args, device)

✅ 成功加载匹配的权重，跳过了形状不符的输出层。


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
our_model.to(device)
for batch_data in dl:
    for key, value in batch_data.items():
        if isinstance(value, torch.Tensor):
            batch_data[key] = value.to(device)

    # change batch_data["image"] shape from (1, C, H, W) to (C,H,W)
    batch_data["image"] = batch_data["image"].squeeze(0)

    output = our_model([batch_data], multimask_output=True)
    print('output masks shape:', output[0]['masks'].shape)

    break



    

I loaded image
after augmentation, image min: 0.0  max: 1.0
x shape:  torch.Size([3, 1024, 1024])
input images:  torch.Size([1, 3, 1024, 1024])


IndexError: index 0 is out of bounds for dimension 1 with size 0

In [ ]:
output[0]["masks"].shape  # (num_masks, H, W)

In [ ]:
a = output[0]["masks"].cpu().numpy()
print('data maxx and minn:', np.max(a), np.min(a))  



定义LOSS函数

In [ ]:
# 配置优化器 
trainable_params = filter(lambda p: p.requires_grad, our_model.parameters())
optimizer = optim.AdamW(trainable_params, lr=1e-4, weight_decay=0.01)

In [ ]:
#定义 Loss
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
#混合精度 Scaler
loss_scaler = GradScaler()

In [ ]:
import os
os.makedirs("models/my_trial", exist_ok=True)
print("文件夹创建成功！")

进行训练

In [ ]:
# -------------------------------------------------------------
# 强制手动定义轮数，不再依赖容易出错的 args
# -------------------------------------------------------------
RUN_EPOCHS = 50   # <--- 我们直接在这里定义，不用管 args 里叫什么了
# -------------------------------------------------------------

# 计算总步数
total_steps = len(dl) * RUN_EPOCHS

print(f" 开始训练... 共 {RUN_EPOCHS} 轮，总计 {total_steps} 步")

our_model.train() # 切换到训练模式

# 创建总进度条
pbar = tqdm(total=total_steps, desc="总体训练进度", unit="step")

for epoch in range(1, RUN_EPOCHS + 1):
    epoch_loss = 0.0
    
    # 内层循环直接遍历 dl
    for step, batch_data in enumerate(dl):
        
        # --- (A) 数据搬运 ---
        images = batch_data["image"].cuda()
        gt_masks = batch_data["mask"].cuda().long().squeeze(1)
        
        batched_input = []
        for i in range(len(images)):
            batched_input.append({
                'image': images[i],
                'original_size': (args.img_size, args.img_size)
            })
            
        # --- (B) 前向传播 ---
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = our_model(batched_input, multimask_output=True)
            pred_masks = torch.stack([o['masks'] for o in outputs]).squeeze(1)
            loss = criterion(pred_masks, gt_masks)
        
        # --- (C) 反向传播 ---
        loss_scaler.scale(loss).backward()
        loss_scaler.step(optimizer)
        loss_scaler.update()
        
        epoch_loss += loss.item()
        
        # 4. 手动更新总进度条
        pbar.update(1) 
        pbar.set_postfix({
            'epoch': f"{epoch}/{RUN_EPOCHS}",
            'loss': f"{loss.item():.4f}"
        })

    # (可选) 打印每轮平均 Loss
    # avg_loss = epoch_loss / len(dl)
    # print(f"Epoch {epoch} 结束 | Avg Loss: {avg_loss:.4f}")
    
    # 保存模型
    if epoch % 5 == 0:
        save_path = os.path.join("models/my_trial", f"model_epoch_{epoch}.pth")
        torch.save(our_model.state_dict(), save_path)

pbar.close()
print("\n 训练全部完成！")

In [ ]:
import Whole_heart_segmentation_junzhe.functions_collection as ff
import numpy as np

# 1. 模拟一个心脏 Mask (在 128x128 空间中)
test_mask = np.zeros((20, 128, 128))
test_mask[5:15, 40:60, 50:80] = 1  # 这是一个长方形的心脏区域

# 2. 调用函数
bbox_info = ff.get_tilted_3d_bbox(test_mask, padding=5)

# 3. 验证结果
if bbox_info:
    x1, y1, x2, y2 = bbox_info['sam_prompt']
    print(f"生成的 BBox: {bbox_info['sam_prompt']}")
    print(f"X宽度: {x2 - x1}, Y宽度: {y2 - y1}") # 应该相等！